In [ ]:
#torch.cuda.empty_cache()
import time
from pysteps.verification.detcatscores import det_cat_fct
from pysteps.verification.detcontscores import det_cont_fct
from pysteps.verification.spatialscores import intensity_scale
from pysteps.visualization import plot_precip_field
device = 'cuda:1'
pcc_average = 0
tokenizer=tokenizer.to(device)
for i, images in enumerate(loaders['test']):
    if i<0:continue
    if i>=5:break

    
    checkpoint = torch.load('/users/zboucher/iris/src/models/tokenizer/vqvae_epoch60', map_location=device)
    optimizer_tokenizer.state_dict()
    image = images[0]
    image = image.unsqueeze(1)
    image = image[3:4, :, :, :]
    a= Variable(image).to(device)  # batch x
        #print(input_image.size())
    a_r = tokenizer.encode_decode(input_image)
    
    
            #checkpoint = torch.load('/users/zboucher/iris/src/models/tokenizer/vqvae_epoch30', map_location = 'cpu')
        #vae.load_state_dict(checkpoint)
        #a_r2 = vae(a)

    for t in range(1):
        #a_display = (a[t,0,:,:].to('cpu').detach().numpy()+1)/2*128
        #a_r_display = (a_r[t,0,:,:].to('cpu').detach().numpy()+1)/2*128
        a_display = a[t,0,:,:].to('cpu').detach().numpy()*40
        a_r_display = a_r[t,0,:,:].to('cpu').detach().numpy()*40
        #a_r1_display = a_r1[t,0,:,:].to('cpu').detach().numpy()*40
        #a_r2_display = a_r2[t,0,:,:].to('cpu').detach().numpy()*40
        
        #np.save('reconstruct_sample{}_gt'.format(i+1), a_display)
        #np.save('reconstruct_sample{}_vqgan1'.format(i+1), a_r_display)
        #np.save('reconstruct_sample{}_vqgan2'.format(i+1), a_r2_display)
        
        scores_cat1 = det_cat_fct(a_r_display, a_display, 1)
        scores_cat2 = det_cat_fct(a_r_display, a_display, 2)
        scores_cat8 = det_cat_fct(a_r_display, a_display, 8)
        scores_cont = det_cont_fct(a_r_display, a_display, thr=0.1)
        #scores_cont2 = det_cont_fct(a_r2_display, a_display, thr=0.1)
        scores_spatial = intensity_scale(a_r_display, a_display, 'FSS', 0.1, [1,10,20,30])
        pcc_average += float(np.around(scores_cont['corr_p'],3))
        if True:
            print('MSE:', np.around(scores_cont['MSE'],3), 
                    'MAE:', np.around(scores_cont['MAE'],3), 
                    'PCC:', np.around(scores_cont['corr_p'],3),'\n', 
                    'CSI(1mm):', np.around(scores_cat1['CSI'],3), # CSI: TP/(TP+FP+FN)
                    'CSI(2mm):', np.around(scores_cat2['CSI'],3),
                    'CSI(8mm):', np.around(scores_cat8['CSI'],3),'\n',
                    'ACC(1mm):', np.around(scores_cat1['ACC'],3), # ACC: (TP+TF)/(TP+TF+FP+FN)
                    'ACC(2mm):', np.around(scores_cat2['ACC'],3),
                    'ACC(8mm):', np.around(scores_cat8['ACC'],3),'\n',
                    'FSS(1km):', np.around(scores_spatial[0][0],3),
                    'FSS(10km):', np.around(scores_spatial[1][0],3),
                    'FSS(20km):', np.around(scores_spatial[2][0],3),
                    'FSS(30km):', np.around(scores_spatial[3][0],3)
                    )  
            plt.figure(figsize=(16, 4))
            plt.subplot(131)
            plot_precip_field(a_display, title="Input")
            plt.subplot(132)
            plot_precip_field(a_r_display, title="Reconstruction_50")
            plt.subplot(133)
            plot_precip_field(a_r1_display, title="Reconstruction_80")
            #plt.subplot(144)
            #plot_precip_field(a_r2_display, title="Reconstruction_80epoch")
            plt.tight_layout()
            plt.show()
print('pcc_average:', pcc_average/i)